In [ ]:
import tensorflow as tf #ver 1.15.0
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('dataset.csv')
df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,DrugC
2,47,M,LOW,HIGH,10.114,DrugC
3,28,F,NORMAL,HIGH,7.798,DrugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,DrugC
196,16,M,LOW,HIGH,12.006,DrugC
197,52,M,NORMAL,HIGH,9.894,DrugX
198,23,M,NORMAL,NORMAL,14.020,DrugX


In [ ]:
#Atribut
features = df[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
label = df[['Drug']].values.reshape(-1,1)

In [ ]:
encoder = OrdinalEncoder()
features = encoder.fit_transform(features)
label = encoder.fit_transform(label)

In [ ]:
#SPLIT DATA
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

In [ ]:
#PREPROCESSING FEATURES
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
#PREPROCESSING LABELS
encoder = OneHotEncoder().fit(y_train)

y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
epochs = 4000
learning_rate = 0.01
n_features = x_train[0].shape[0]
n_classes = y_train[0].shape[0]

In [ ]:
#neural network
input_tensor = tf.placeholder(tf.float32, name = 'input')
label_tensor = tf.placeholder(tf.float32, name = 'label')

In [ ]:
neurons = [5, 64, 5]
n_layers = len(neurons)

In [ ]:
parameters = {
    'W1' : tf.Variable(
        tf.random.normal([
            n_features, neurons[0]
        ]),
        tf.float32
    ),
    'B1' : tf.Variable(
        tf.random.normal([1,  neurons[0]]),
        tf.float32
    ),
    'W2' : tf.Variable(
        tf.random.normal([
            neurons[0], neurons[1]
        ]),
        tf.float32
    ),
    'B2' : tf.Variable(
        tf.random.normal([1,  neurons[1]]),
        tf.float32
    ),
    'W3' : tf.Variable(
        tf.random.normal([
            neurons[1], neurons[2]
        ]),
        tf.float32
    ),
    'B3' : tf.Variable(
        tf.random.normal([1,  neurons[2]]),
        tf.float32
    )
}

In [ ]:
#FORWARD PASS
def forward_pass(input_tensor,params,n_layer=3):
  a = input_tensor
  for i in range(n_layer):
    W = params[f'W{i+1}']
    B = params[f'B{i+1}']

    z = tf.matmul(a,W) + B 
    if i == n_layer-1:
      a = tf.nn.sigmoid(z)
    else:
      a = tf.nn.sigmoid(z)

  return a 

In [ ]:
def evaluate_accuracy(y, yhat):
  yield (0.5 * (yhat - y)**2).sum(axis=1).mean()
  yield accuracy_score(np.argmax(y, axis = 1), np.argmax(yhat, axis = 1))

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward_pass(input_tensor, parameters)
  loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_tensor)
  sess.run(tf.global_variables_initializer())
  for i in range(epochs):
    sess.run(
        optimiser,
        feed_dict = {
            input_tensor: x_train,
            label_tensor: y_train
        }
    )

    if i%200 == 0:
      loss,acc = evaluate_accuracy(y_test,sess.run(
          logits_tensor,
          feed_dict = {
              input_tensor: x_test,
              label_tensor: y_test
          }
      ))
      print(f'Epochs: ',i, ' ||Loss: ',loss, ' ||Accuracy: ', {acc*100},'%')

Epochs:  0  ||Loss:  1.4163835  ||Accuracy:  {7.5} %
Epochs:  200  ||Loss:  1.2803164  ||Accuracy:  {5.0} %
Epochs:  400  ||Loss:  1.2012577  ||Accuracy:  {2.5} %
Epochs:  600  ||Loss:  1.1523129  ||Accuracy:  {0.0} %
Epochs:  800  ||Loss:  1.0462433  ||Accuracy:  {0.0} %
Epochs:  1000  ||Loss:  0.8834484  ||Accuracy:  {0.0} %
Epochs:  1200  ||Loss:  0.8254644  ||Accuracy:  {0.0} %
Epochs:  1400  ||Loss:  0.80113614  ||Accuracy:  {0.0} %
Epochs:  1600  ||Loss:  0.7721357  ||Accuracy:  {5.0} %
Epochs:  1800  ||Loss:  0.7258875  ||Accuracy:  {5.0} %
Epochs:  2000  ||Loss:  0.6543785  ||Accuracy:  {5.0} %
Epochs:  2200  ||Loss:  0.5422106  ||Accuracy:  {12.5} %
Epochs:  2400  ||Loss:  0.40864247  ||Accuracy:  {25.0} %
Epochs:  2600  ||Loss:  0.31899554  ||Accuracy:  {50.0} %
Epochs:  2800  ||Loss:  0.27593604  ||Accuracy:  {70.0} %
Epochs:  3000  ||Loss:  0.25498202  ||Accuracy:  {72.5} %
Epochs:  3200  ||Loss:  0.24204692  ||Accuracy:  {72.5} %
Epochs:  3400  ||Loss:  0.23257633  ||Accur